# Tutorial (6): QITE and Multistate calculation

Here we show how to run QITE especially with a model space

[(1) Classical imaginary-time-evolution](#(1)-Classical-imaginary-time-evolution)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Initializing CITE](#Initializing-CITE)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Running CITE](#Running-CITE)  

[(2) UCCGSD-based QITE](#(2)-UCCGSD-based-QITE)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Initializing QITE](#Initializing-QITE)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Running QITE](#Running-QITE)  

[(3) Model-Space QITE](#(3)-Model-Space-QITE)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Initializing MSQITE](#Initializing-MSQITE)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Example of structure of `multi`](#Example-of-structure-of-multi)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[Running MSQITE](#Running-MSQITE)  




In [1]:
# import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf

mpi4py is not imported. no MPI.


## (1) Classical imaginary-time-evolution
Setting `method = qite` updates the state based on imaginary-time-evolution:
\begin{equation}
|\Phi^{(k+1)}\rangle = \frac{e^{-\Delta\beta \hat H} |\Phi^{(k)}\rangle}{\langle \Phi^{(k)} | e^{-2\Delta\beta \hat H} |\Phi^{(k)}\rangle}  
\end{equation}
First, we consider to perform the above non-unitary operation numerically exactly by `ansatz = cite` (classical ITE).  
$\Delta\beta$ is set with `dt`, `db`, or `timestep`.

### Initializing CITE

In [2]:
### Create QuketData for H4 example ###
CITE = create(
    basis        = 'sto-6G',
    multiplicity = 1,
    charge       = 0,
    n_electrons  = 4,
    n_orbitals   = 4,
    geometry     = "H  0  0  0    ;\
                    H  0  0  1.2  ;\
                    H  1  0  0    ;\
                    H  1  0  1.2 ",
    method       = 'qite',
    ansatz       = 'cite',
    dt           = 0.2,
    maxiter      = 1000,
    det          = '00001111',
    taper_off    = True,
    run_qubitfci = True
)

Basis set = sto-6G

*** Geometry ******************************
  H     0.0000000    0.0000000    0.0000000
  H     0.0000000    0.0000000    1.2000000
  H     1.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    1.2000000
*******************************************

Symmetry D2h : D2h(Abelian)
E[FCI]    = -2.033812422170     (Spin = 1   Ms = 0)
E[HF]     = -1.939420943306     (Spin = 1   Ms = 0)
TODO: Pauli_list is not supported for ansatz = cite.
Tapering-Off Results:
List of Tapered-off Qubits:  [0, 1, 2, 4]
Qubit: 0    Tau: 1.0 [Z0 Z3 Z5 Z6]
Qubit: 1    Tau: 1.0 [Z1 Z3 Z5 Z7]
Qubit: 2    Tau: 1.0 [Z2 Z3 Z6 Z7]
Qubit: 4    Tau: 1.0 [Z4 Z5 Z6 Z7]

States     transformed.


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


Operators  transformed.
NBasis = 4
States     backtransformed.
Operators  backtransformed.
States     transformed.
Operators  transformed.
FCI in Qubits (tapered-off mapping)
(FCI state : E = -2.0338124221702545)
  Basis          Coef
| 0001 > : +0.9391 +0.0000i
| 0010 > : -0.2519 +0.0000i
| 0111 > : -0.1083 +0.0000i
| 1000 > : -0.1083 +0.0000i



### Running CITE

In [3]:
CITE.run()

Enetered QITE driver
Performing QITE for chemical Hamiltonian
Ansatz = cite
Initial configuration: | 00001111 >
Convergence criteria: ftol = 1E-09
ITE
Shift =  -1.9394209433056588
QLanczos =  False
  0.00: E = -1.939420943306  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.881940  CPU Time =  0.00
  0.20: E = -1.970402587074  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.909779  CPU Time =  0.08
  0.40: E = -1.989923096551  <S**2> = -0.00000000  <N> = 4.00000000  Fidelity = 0.929241  CPU Time =  0.08
  0.60: E = -2.002447020364  <S**2> = -0.00000000  <N> = 4.00000000  Fidelity = 0.943163  CPU Time =  0.08
  0.80: E = -2.010667922113  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.953395  CPU Time =  0.07
  1.00: E = -2.016206677251  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.961130  CPU Time =  0.08
  1.20: E = -2.020044730614  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.967136  CPU Time =  0.07
  1.40: E = -2.022782899109  <S**2> = -0.00000000  <N

## (2) UCCGSD-based QITE
Setting `ansatz` to a pre-defined ansatz in `qite_ansatz_list`, `config.py`, prepares an operator pool.  
Here, we set `ansatz = uccgsd`.

Also, change the initial state to `0.9391 * |00001111> - 0.2518 * |00110011>` to accelerate the convergence.

#### Initializing QITE

In [4]:
QITE = CITE.copy()
QITE.set(ansatz = 'uccgsd', det="0.9391 * |00001111> - 0.2518 * |00110011>")
QITE.initialize()

Basis set = sto-6G

*** Geometry ******************************
  H     0.0000000    0.0000000    0.0000000
  H     0.0000000    0.0000000    1.2000000
  H     1.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    1.2000000
*******************************************

Symmetry D2h : D2h(Abelian)
E[FCI]    = -2.033812422170     (Spin = 1   Ms = 0)
E[HF]     = -1.939420943306     (Spin = 1   Ms = 0)
Tapering-Off Results:
List of Tapered-off Qubits:  [0, 1, 2, 4]
Qubit: 0    Tau: 1.0 [Z0 Z3 Z5 Z6]
Qubit: 1    Tau: 1.0 [Z1 Z3 Z5 Z7]
Qubit: 2    Tau: 1.0 [Z2 Z3 Z6 Z7]
Qubit: 4    Tau: 1.0 [Z4 Z5 Z6 Z7]

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.


#### Running UCCGSD-based QITE

In [5]:
QITE.run()

Enetered QITE driver
Performing QITE for chemical Hamiltonian
Ansatz = uccgsd
Initial configuration:  +0.9391+0.0000j * | 00001111 > -0.2518+0.0000j * | 00110011 >
Convergence criteria: ftol = 1E-09
QITE: Pauli operator group size = 30
Timing for preparing nonredundant sigma list: 0.0 sec
    Unique sigma list = 71
Shift =  -1.9596549885611783
QLanczos =  False
  0.00: E = -1.959654988561  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.00
  0.20: E = -1.994102853769  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.03
  0.40: E = -2.012484390895  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.02
  0.60: E = -2.022225735883  <S**2> = +0.00000001  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.02
  0.80: E = -2.027415162393  <S**2> = +0.00000001  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.02
  1.00: E = -2.030212805871  <S**2> = +0.00000001  <N> = 4.00000000  Fidelity = 0.000000  CPU Time = 

## (3) Model-Space QITE 
MSQITE is invoked by `msqite = True` with `method = qite`.

Some algorithms including MSQITE need multiple `state`s and `init_state`s.  
These states are defined by `multi` option and handled by the `multi` subclass.  

`multi` option requires a list of state descriptions. 
Each state description is given by a tuple (or list) of initial state (single determinant or multi determinants) and weight
```python
   multi = [(det1, weight1), (det2, weight2)]
```
Here, weight1, weight2, ... are a positive number for state-averaging (thus will be normalized),  
and det1, det2, ... are specified in the same way as `det`.

For MSQITE, weights play no role.

#### Initializing MSQITE

In [6]:
MSQITE = QITE.copy()
MSQITE.set(multi=[("|00001111>", 1), ("0.6 *|00110011> - 0.3 *|11000011>", 1)], msqite=True)
MSQITE.initialize()

Basis set = sto-6G

*** Geometry ******************************
  H     0.0000000    0.0000000    0.0000000
  H     0.0000000    0.0000000    1.2000000
  H     1.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    1.2000000
*******************************************

Symmetry D2h : D2h(Abelian)
E[FCI]    = -2.033812422170     (Spin = 1   Ms = 0)
E[HF]     = -1.939420943306     (Spin = 1   Ms = 0)
Existing theta_list is inconsistent :
   size of pauli_list 162 != size of theta_list 30.
Ovewrite theta_list by zero.
Tapering-Off Results:
List of Tapered-off Qubits:  [0, 1, 2, 4]
Qubit: 0    Tau: 1.0 [Z0 Z3 Z5 Z6]
Qubit: 1    Tau: 1.0 [Z1 Z3 Z5 Z7]
Qubit: 2    Tau: 1.0 [Z2 Z3 Z6 Z7]
Qubit: 4    Tau: 1.0 [Z4 Z5 Z6 Z7]

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.


#### Example of structure of `multi`
`init_state_info` contains initial configurations: single-determinants/multi-determinants

In [7]:
print(MSQITE.multi.init_states_info)
# Base-10 integer = Determinant"

[15, [[(0.6+0j), 51], [(-0.3+0j), 195]]]


This result means we have two configurations,
```python
15
```
and 
```python
[[(0.6+0j), 15], [(-0.3+0j), 195]]
```
Single determinants are represented by a base-10 integer.  
A multi determinant description is given by a list of tuples of (`coefficient`, `determinant` (base-10))

In [8]:
for k, (state, weight) in enumerate(zip(MSQITE.multi.states, MSQITE.multi.weights)):
    print_state(state, f"{k}-th state in multi. Weight = {weight}")

0-th state in multi. Weight = 1.0
  Basis          Coef
| 0001 > : +1.0000 +0.0000i

1-th state in multi. Weight = 1.0
  Basis          Coef
| 0010 > : +0.8944 +0.0000i
| 1100 > : -0.4472 +0.0000i



#### Running MSQITE

In [9]:
MSQITE.run()

Enetered QITE driver
Performing QITE for chemical Hamiltonian
Ansatz = uccgsd
Initial configuration:  +0.9391+0.0000j * | 00001111 > -0.2518+0.0000j * | 00110011 >
Convergence criteria: ftol = 1E-09
MSQITE (State-Specific): Pauli operator group size = 30
Timing for preparing nonredundant sigma list: 0.0 sec
    Unique sigma list = 71
Initial state 0
  Basis          Coef
| 0001 > : +1.0000 +0.0000i

Initial state 1
  Basis          Coef
| 0010 > : +0.8944 +0.0000i
| 1100 > : -0.4472 +0.0000i

Shift =  [(-1.9394209433056593+0j), (-1.5655253081245326+0j)]
QLanczos =  False
  0.00
      state0: E = -1.939787627916  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.01  
      state1: E = -1.565158623514  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.01  

  0.20
      state0: E = -1.977488376452  <S**2> = +0.00000000  <N> = 4.00000000  Fidelity = 0.000000  CPU Time =  0.07  
      state1: E = -1.621671913419  <S**2> = +0.00000000  <N> =